In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# 데이터셋의 목록 읽어오기
file_list = os.listdir("./Dataset")
file_list

['a.npy']

In [3]:
# 파일 확장자 빼고 제스처 이름만 저장
actions = []
for i in range(len(file_list)) :
    actions.append(file_list[i][:-4])

actions

['a']

In [4]:
# 각 축을 기준으로 회전변환할 각도 Rotation Degree

## x축 기준 회전각 : pitch
RDx = np.deg2rad([5,10,20,30, 355,350,340,330])
## y축 기준 회전각 : yaw
RDy = np.deg2rad([5,10,20,30,40, 355,350,340,330,320])
## z축 기준 회전각 : roll
RDz = np.deg2rad([20,40,60,80,100,120,140,160,180,200,220,240,260,280,300,320,340])

In [5]:
RDx

array([0.08726646, 0.17453293, 0.34906585, 0.52359878, 6.19591884,
       6.10865238, 5.93411946, 5.75958653])

In [6]:
# 회전변환 행렬
def Rx(theta) :
    return np.array([ [1, 0, 0], 
                    [0, np.cos(theta), -np.sin(theta)], 
                    [0, np.sin(theta), np.cos(theta)]])
def Ry(theta) :
    return np.array([[np.cos(theta), 0, np.sin(theta)], 
                    [0, 1, 0],
                    [-np.sin(theta), 0, np.cos(theta)]])
def Rz(theta) :
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                    [np.sin(theta), np.cos(theta), 0], 
                    [0, 0, 1]])

### 연습

In [7]:
loaded = np.load(f'./Dataset/{actions[i]}.npy')
# loaded

In [8]:
len(loaded)

17

In [9]:
loaded[0][0]

array([ 7.29140759e-01,  8.69784117e-01, -2.36627820e-08])

In [10]:
loaded[0][0] @ Rz(np.deg2rad(90)) @ Ry(np.deg2rad(90)) @ Rx(np.deg2rad(90))

array([2.36627820e-08, 8.69784117e-01, 7.29140759e-01])

In [11]:
np.dot( np.dot( np.dot( loaded[0][0], Rz(np.deg2rad(90)) ), Ry(np.deg2rad(90)) ), Rx(np.deg2rad(90)) )

array([2.36627820e-08, 8.69784117e-01, 7.29140759e-01])

In [12]:
a = loaded[:, [5,   1,2,3,   5,6,7,   9,10,11,   13,14,15,   17,18,19,   0,0], :]
b = loaded[:, [17,   2,3,4,   6,7,8,   10,11,12,   14,15,16,   18,19,20,   5,17], :] 
v = b - a
v.shape, v

((17, 18, 3),
 array([[[ 1.24974132e-01,  5.95875978e-02, -1.68120842e-02],
         [-5.07960320e-02, -7.36625791e-02, -1.49487853e-02],
         [-2.72036791e-02, -6.22540712e-02, -1.36239454e-02],
         [-2.48087049e-02, -4.74437475e-02, -1.27459131e-02],
         [-8.37218761e-03, -9.47859287e-02, -2.08203923e-02],
         [ 1.00672245e-04, -5.28311729e-02, -1.30215809e-02],
         [ 1.60217285e-03, -4.56566215e-02, -1.04239322e-02],
         [ 1.90329552e-02, -7.35141039e-02, -4.19993326e-02],
         [ 1.43730044e-02,  5.96137047e-02, -1.37612298e-02],
         [ 3.02016735e-04,  5.65192699e-02,  1.23430789e-03],
         [ 2.34279037e-02, -5.70072532e-02, -4.24760059e-02],
         [ 7.92425871e-03,  5.59706092e-02, -1.42179430e-04],
         [-5.87809086e-03,  4.96908426e-02,  1.08543783e-02],
         [ 2.18430758e-02, -4.08098698e-02, -3.30022164e-02],
         [ 2.45809555e-04,  3.89668941e-02,  1.91517174e-04],
         [-9.12052393e-03,  3.93981934e-02,  7.84572959e

In [13]:
unitv = v / np.linalg.norm(v, axis=2)[:,:,np.newaxis]
unitv.shape, unitv

((17, 18, 3),
 array([[[ 8.96065059e-01,  4.27243332e-01, -1.20542716e-01],
         [-5.59929145e-01, -8.11989112e-01, -1.64781780e-01],
         [-3.92601338e-01, -8.98445816e-01, -1.96619699e-01],
         [-4.50781387e-01, -8.62066699e-01, -2.31596951e-01],
         [-8.59513405e-02, -9.73100223e-01, -2.13748271e-01],
         [ 1.85017249e-03, -9.70940727e-01, -2.39312938e-01],
         [ 3.41914631e-02, -9.74343489e-01, -2.22453834e-01],
         [ 2.19327818e-01, -8.47145799e-01, -4.83982750e-01],
         [ 2.28698249e-01,  9.48552539e-01, -2.18963904e-01],
         [ 5.34225640e-03,  9.99747354e-01,  2.18331915e-02],
         [ 3.12987370e-01, -7.61593975e-01, -5.67462355e-01],
         [ 1.40180518e-01,  9.90122768e-01, -2.51516095e-03],
         [-1.14804068e-01,  9.70504034e-01,  2.11995157e-01],
         [ 3.84235499e-01, -7.17875121e-01, -5.80532852e-01],
         [ 6.30796221e-03,  9.99968027e-01,  4.91471171e-03],
         [-2.21403437e-01,  9.56402890e-01,  1.90457425e

In [17]:
# 내적을 이용한 각도 계산 ( a•b = |a||b|cos(Θ) )    # 내적, cos의 역수
angle = np.arccos( np.einsum('nt,nt->n',
                                unitv[:, [0,1,2,  0,4,5,  0,7,8,  0,10,11,   0,13,14,   3,6,9,12,15,   6,9,12,15], :], 
                                unitv[:, [1,2,3,  4,5,6,  7,8,9,  10,11,12,   13,14,15,   0,0,0,0,0,   2,2,2,2,2], :]) 
)
angle.shape, angle

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (17,24,3)->(17,24,3) (17,25,3)->(17,25,3) 

### 좌표데이터 회전변환

In [ ]:
# 각 제스처 좌표데이터별로
for i, gest in enumerate(actions):
    loaded = np.load(f'./Dataset/{actions[i]}.npy')
    
    # 회전변환
    gestCoord = []
    for rdz in RDz :
        for rdy in RDy :
            for rdx in RDx :
                
                for i in range(len(loaded)) :
                    frame = []
                    for j in range(21) :
                        frame.append( loaded[i][j] @ Rz(rdz) @ Ry(rdy) @ Rx(rdx) )
                        
                gestCoord.append((frame))

In [ ]:
gestCoord

In [ ]:
a = np.array(gestCoord)
a.shape , a

### 좌표데이터 회전변환 + 각도데이터로 변환 + 시퀀스데이터로 변환

In [ ]:
# 각 제스처 좌표데이터별로
for i, gest in enumerate(actions):
    loaded = np.load(f'./Dataset/{actions[i]}.npy')
    
    
    
    # 회전변환
    gestCoord = []
    for rdz in RDz :
        for rdy in RDy :
            for rdx in RDx :
                
                for i in range(len(loaded)) :    # 프레임별
                    frame = []
                    for j in range(21) :    # 랜드마크별
                        
                        
                        
                        frame.append( loaded[i][j] @ Rz(rdz) @ Ry(rdy) @ Rx(rdx) )
                        
                gestCoord.append((frame))